### Semantic Kernel Examples

In [20]:
import os

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextCompletion, OpenAIChatCompletion
from semantic_kernel.functions import kernel_function
from semantic_kernel import Kernel
from semantic_kernel.core_plugins.time_plugin import TimePlugin
from semantic_kernel.core_plugins.conversation_summary_plugin import ConversationSummaryPlugin
from semantic_kernel.prompt_template import PromptTemplateConfig, InputVariable
from semantic_kernel.functions import KernelArguments
from semantic_kernel.functions.kernel_plugin import KernelPlugin

load_dotenv()

api_key = os.getenv("AZURE_OPENAI_API_KEY")
base_url = os.getenv("AZURE_OPENAI_API_ENDPOINT")
ai_model_id = os.getenv("DEPLOYMENT_NAME")


In [21]:
kernel = Kernel()

kernel.remove_all_services()

chat_completion_service = AzureChatCompletion(
    service_id="azure-gpt35",
    deployment_name=ai_model_id,
    api_key=api_key,
    endpoint=base_url,
)

kernel.add_service(chat_completion_service)

# chat_completion_service = OpenAIChatCompletion(
#     service_id="openai-gpt35",
#     ai_model_id="gpt-3.5-turbo",  # or "gpt-4"
#     api_key=api_key,
# )

# kernel.add_service(chat_completion_service)

# text_completion_service = AzureTextCompletion(
#     service_id="azure-gpt35-text", 
#     deployment_name=ai_model_id,
#     api_key=api_key,
#     endpoint=base_url,
# )

# kernel.add_service(text_completion_service)


In [ ]:

agent = ChatCompletionAgent(
    kernel=kernel,
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations.",
)

async def main():
    thread: ChatHistoryAgentThread | None = None

    user_input = input("Enter your travel destination: ")
    messages = []

    while user_input.lower() != "exit":
        print(f"You: {user_input}")
        messages.append(user_input)
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=messages,
            thread=thread,
        ):
            if first_chunk:
                print(f"Agent: {response}", end="", flush=True)
                first_chunk = False
            
            print(response, end="", flush=True)
            messages.append(response.content)
            thread = response.thread
        print()
        # Get the next user input
        user_input = input("You: ")
    
    await thread.delete() if thread else None


await main()


You: Kashmir
Agent: Kashmir is a beautiful region located in the northern part of India, known for its stunning natural beauty, including snow-capped mountains, lush green valleys, and serene lakes. Here are some recommendations for your vacation to Kashmir:

1. **Srinagar**: The summer capital of Jammu and Kashmir, Srinagar is famous for its breathtaking Dal Lake, houseboats, and Mughal gardens like Shalimar Bagh and Nishat Bagh.

2. **Gulmarg**: Known as the "Meadow of Flowers," Gulmarg is a popular hill station and a paradise for nature lovers and adventure enthusiasts. You can enjoy activities like skiing, trekking, and golfing here.

3. **Pahalgam**: This picturesque town is surrounded by lush forests, pristine rivers, and meadows. It is a perfect destination for those seeking tranquility and outdoor adventures like hiking and fishing.

4. **Sonamarg**: Also known as the "Meadow of Gold," Sonamarg is a scenic valley located at an altitude of 2740 meters. It offers stunning views o

In [11]:
time_plugin = TimePlugin()
kernel.add_plugin(time_plugin, plugin_name="time")


prompt_template_config = PromptTemplateConfig(
    name="SummarizeConversation",
    template="Summarize the following conversation:\n{{$input}}",
    description="Summarize a conversation into a few key points.",
        input_variables=[InputVariable(name="input", description="The conversation text", type="string")]
)

conversation_summary_plugin = ConversationSummaryPlugin(prompt_template_config=prompt_template_config)
kernel.add_plugin(conversation_summary_plugin, plugin_name="conversation_summary")

p = kernel.plugins["conversation_summary"]
# print(f"Plugin loaded: {p.name} - {p.dict()}")

async def main():
    # result = await kernel.invoke(plugin_name="time", function_name="dayOfWeek")
    # print(f"Current time: {result}")

    # prompt = input("Enter a prompt for the conversation summary: ")
    # print(f"Prompt: {prompt}")
    # args = KernelArguments(input=prompt)
    # summary = await kernel.invoke(
    #     plugin_name="conversation_summary",
    #     function_name="SummarizeConversation",
    #     arguments=args
    # )
    # print(f"Conversation Summary: {summary}")

    prompt = """
    This is some information about the user's background: {{$careerHistory}}
    Given the user's background, provide a list of relevant career choices.
    """
    career_history = input("Enter your backgroung: ")
    print(f"CareerHistory: {career_history}")
    args = KernelArguments(careerHistory=career_history)

    career_choices = await kernel.invoke_prompt(
        prompt=prompt,
        arguments=args,
    )

    print(f"Career Choices: {career_choices}")

await main()


CareerHistory: Botanist
Career Choices: 1. Botanical researcher
2. Environmental consultant specializing in plant ecosystems
3. Plant conservation specialist
4. Agricultural scientist focusing on plant genetics or crop improvement
5. Plant breeder or nursery manager
6. Ecologist studying plant interactions in ecosystems
7. Botanical illustrator or photographer
8. Horticulturist specializing in rare or endangered plant species
9. Botanical garden curator or educator
10. Field biologist conducting plant surveys or biodiversity assessments.


In [46]:
career_focus = input("Enter your career focus: ")

prompt = f"""
    The following is a conversation between a user and a coaching assistant.
    The assistant is helpful, creative, clever, and very friendly.

    <message role="user">Can you give me some career suggestions, since I want to be able to
    travel more?</message>
    <message role="assistant">Of course! What is your current career or area of expertise?</message>
    <message role="user">{career_focus}.</message>
    """

# print(f"Prompt: {prompt}")

result = await kernel.invoke_prompt(
    prompt=prompt,

)
print(f"Career Suggestions: {result}")

Career Suggestions: That's awesome! Here are a few career suggestions that might allow you to combine your passion for cycling with your desire to travel more:

1. Professional Cyclist: Consider pursuing a career as a professional cyclist, competing in races and events around the world.

2. Cycling Coach: Use your expertise in cycling to become a coach, leading training camps and clinics in different locations.

3. Cycling Tour Guide: Become a cycling tour guide, leading groups on biking tours through different countries and regions.

4. Cycling Blogger/Influencer: Start a blog or social media account focused on cycling and travel, sharing your adventures and tips with a global audience.

5. Bicycle Mechanic: Work as a bicycle mechanic at cycling events or in different cities, allowing you to travel while also using your technical skills.

I hope these suggestions help spark some ideas for your career path!


In [ ]:


kernel.add_plugin(parent_directory="./semantic_plugins", plugin_name="career_coach")


p = kernel.plugins["career_coach"]
print(f"Plugin loaded: {p.name} - {p.model_dump()}")


career_focus = input("Enter your career focus: ")

print(f"Career Focus: {career_focus}")

result = await kernel.invoke(
    plugin_name="career_coach",
    function_name="get_career_advice",
    arguments=KernelArguments(career_focus=career_focus),
)
print(f"Career Focus Result: {result}")


In [ ]:
from native_plugins.career_history.career_tracker_plugin import CareerTrackerPlugin

career_tracker_plugin = CareerTrackerPlugin()

# kernel.add_plugin(career_tracker_plugin, plugin_name="career_tracker")
# ct = kernel.plugins["career_tracker"]
# print(f"Plugin loaded: {ct.name} - {ct.model_dump()}")

# result = await kernel.invoke(plugin_name="career_tracker", function_name="get_career_history")
result = career_tracker_plugin.get_career_history()
print(f"Result {result}")
